##### Copyright 2024 Google LLC.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Function calling with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemini-guide/blob/main/quickstart/function_calling_python.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/gemini-guide/blob/main/quickstart/function_calling_python.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

You can provide Gemini models with descriptions of functions. The model may ask you to call a function and send it back the result to help the model handle your query.

## Setup


### Install the Python SDK

The Python SDK for the Gemini API is contained in the [`google-generativeai`](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip:


In [2]:
!pip install -U -q google-generativeai

### Import packages

Import the necessary packages.

In [3]:
import google.generativeai as genai

### Set up your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`


In [4]:
import os
try:
    # Used to securely store your API key
    from google.colab import userdata
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
except ImportError:
    pass    


## Function Basics

You can pass a list of functions to the `tools` argument when creating a `genai.GenerativeModel`.

> Important: The SDK converts the function's argument's type annotations to a format the API understands (`glm.FunctionDeclaration`). The API only supports a limited selection of argument types, and this automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [5]:
def add(a:float, b:float):
    """returns a + b."""
    return a+b

def subtract(a:float, b:float):
    """returns a - b."""
    return a*b

def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

def divide(a:float, b:float):
    """returns a / b."""
    return a*b

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[add, subtract, multiply, divide])

model

genai.GenerativeModel(
    model_name='models/gemini-1.0-pro',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x109ac0c50>,
)

The recomended way to use function calling is through the chat interface. The main reason is that `FunctionCalls` fit nicely into chat's multi-turn structure.

In [6]:
chat = model.start_chat(enable_automatic_function_calling=True)

With automatic function calling enabled `chat.send_message` automatically calls your function if the model asks it to.

It appears to simply return a text response, containing the correct answer:

In [7]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'There are 2508 mittens in total.'

In [8]:
57*44

2508

If you look in the `ChatSession.history` you can the sequence of `glm.Content` objects representing the turns of the conversation.
Each `glm.Content` contains a list of `glm.Part` objects that either contain text, a `glm.FunctionCall` or a `glm.FunctionResponse`.

Here the sequence of turns is:

1. You sent the question.
2. The model replied with a `glm.FunctionCall`.
3. The `genai.ChatSession` executed the function locally and sent the model back a `glm.FunctionResponse`.
4. The model used the function output in its answer.

In [9]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print('-'*80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 2508 mittens in total.'}]
--------------------------------------------------------------------------------


In general the state diagram is:

<img src="https://ai.google.dev/images/tutorials/function_call_state_diagram.png" alt="The model can always reply with text, or a FunctionCall. Iff the model sends a FunctionCall the user must reply with a FunctionResponse" width=50%>

The model can respond with multiple function calls before returning a text response, and function calls come before the text response.

While this was all handled automatically, if you need more control, you can:

- Leave the default `enable_automatic_function_calling=False` and process the `glm.FunctionCall` responses yourself.
- Or use `GenerativeModel.generate_content`, where you also need to manage the chat history. 

## Another example

On the FunctionCalling overview page, there's a [movie finding example](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample), here's the rough equivalent, in python:

In [7]:
def find_movies(description: str, location: str = ''):
  """find movie titles currently playing in theaters based on any description, genre, title words, etc.

  Args:
      description: Any kind of description including category or genre, title words, attributes, etc.
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
  """
  pass

def find_theaters(location: str, movie: str = ''):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.
    
    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    pass

def get_showtimes(location:str, movie:str, theater:str, date:str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    pass

model = genai.GenerativeModel(model_name='gemini-1.0-pro',
                              tools=[find_movies, find_theaters, get_showtimes])


Ask the model in natural languag and it returns a function call:

In [10]:
response = model.generate_content("Which theaters in Mountain View show the Barbie movie?")
response.candidates[0].content.parts

[function_call {
  name: "find_theaters"
  args {
    fields {
      key: "movie"
      value {
        string_value: "Barbie"
      }
    }
    fields {
      key: "location"
      value {
        string_value: "Mountain View, CA"
      }
    }
  }
}
]

## Next Steps

Useful API references:

- The [genai.GenerativeModel](https://ai.google.dev/api/python/google/generativeai/GenerativeModel) class
  - Its [GenerativeModel.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) method builds a [glm.GenerateContentRequest](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentRequest) behind the scenes.
    - The request's `.tools` field contains a list of 1 [glm.Tool](https://ai.google.dev/api/python/google/ai/generativelanguage/Tool) object.
    - The tool's `function_declarations` attribute contains a list of [FunctionDeclarations](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionDeclaration) objects.
- The [response](https://ai.google.dev/api/python/google/ai/generativelanguage/GenerateContentResponse) may contain a [glm.FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall), in `response.candidates[0].contents.parts[0]`.
- if `enable_automatic_function_calling` is set the [genai.ChatSession](https://ai.google.dev/api/python/google/generativeai/ChatSession) executes the call, and sends back the [glm.FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse).
- In response to a [FunctionCall](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall) the model always expects a [FunctionResponse](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse).
- If you reply manually using [chat.send_message](https://ai.google.dev/api/python/google/generativeai/ChatSession#send_message) or [model.generate_content](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#generate_content) remember thart the API is stateless you have to send the whole conversation history (a list of [content](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) objects), not just the last one containing the `FunctionResponse`.